* Let's analyze the HR Analytics dataset by performing EDA & find out useful insights & information which would help us to predict that which all employees   might be searching for a new job.

* Later, Let's Train different models & compare their metrics.

In [ ]:
# Importing the required libraries


import numpy as np # linear algebra
import pandas as pd # data processing


#visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


from datetime import datetime
from scipy.stats import skew
import time
import copy
import math

from sklearn.feature_selection import SelectKBest,chi2,f_classif,mutual_info_classif  # Feature Engineering
from sklearn.model_selection import train_test_split                                  # Splitting the dataset into training & testing
from imblearn.over_sampling import SMOTE                                              # For Handling Data Imbalance
from sklearn.preprocessing import OrdinalEncoder                                      # Label Encoding library 
from sklearn.utils import resample                                                    # Data Resampling Library


# Regression & Classification Models
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from skopt import BayesSearchCV


#from tune_sklearn import TuneSearchCV
from sklearn.linear_model import RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


# Model Metric libraries
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score,recall_score,precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve

In [ ]:
# Importing the dataset

df_train=pd.read_csv("../input/hr-analytics-job-change-of-data-scientists/aug_train.csv")
df_test=pd.read_csv("../input/hr-analytics-job-change-of-data-scientists/aug_test.csv")

In [ ]:
df_train.head()

In [ ]:
print("Training Dataset Info:-")
print("\n")
print(df_train.info())

In [ ]:
#Checking for missing data

print("Missing Data in the Training Dataset:-")
print("\n")
print(df_train.isnull().sum())

print("\n")

#Checking for missing data (In Percentage(%))

print("Missing Data in the Training Dataset (In %):-")
print("\n")
print((df_train.isnull().sum()/len(df_train))*100)

* **As there are missing values present in both Train & Test datasets, we need to replace the missing values with more suitable & efficient values.**
* **If we replace missing values with mean or median, it affects the data distribution & may affect the Model performance during model building.**
* **So, it's best to use "ffill" & "bfill" methods as the data will be distributed evenly without affecting the distribution.**

In [ ]:
# Let's drop the Missing values.

missing_value_fields=['gender','enrolled_university','education_level','major_discipline','experience','company_size','company_type',
                      'last_new_job']

df_train=df_train.dropna()

print("Training Dataset:-")

print("\n")
print(df_train.isnull().sum())

# Data Visualization & EDA

**Let's analyze the relationship of each feature with the target variable with the help of crosstab to gain insights.**

In [ ]:
#Gender

df_train.gender.value_counts()

In [ ]:
# Dependancy of Gender on Target Variable

Gender_crosstab=pd.crosstab(index=df_train['gender'],columns=df_train['target']).sort_values(by=1.0,ascending=False)
Gender_crosstab["Job Changers' Percentage (%)"]=round(Gender_crosstab[1]/(Gender_crosstab[0]+Gender_crosstab[1])*100,2)
print(Gender_crosstab)


# Let's plot the result and visualize.

g=Gender_crosstab.drop(["Job Changers' Percentage (%)"], axis=1).plot(kind='bar',stacked=False,figsize=(20,6))
for p in g.patches:
    g.annotate(format(p.get_height()), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center',
                   fontsize=12,
                   xytext = (0, 5), 
                   textcoords = 'offset points',
                   rotation='horizontal')
plt.xticks(rotation = 0,fontsize=14)
plt.yticks(fontsize=12)
plt.xlabel('Gender',fontsize=16)
plt.ylabel('Target',fontsize=16)
plt.title('HR Analysis w.r.t Gender',fontsize=18)
plt.legend(['Not Looking for a Job','Looking for a Job'],title='Job Change Likelihood',title_fontsize=16,prop={"size":14})

* Most of the job searchers are in the male category followed by the females and others.

In [ ]:
#Revlevant Experience

df_train.relevent_experience.value_counts()

In [ ]:
# Dependancy of Relevant Experience on Target Variable

Relevant_Exp_crosstab=pd.crosstab(index=df_train['relevent_experience'],columns=df_train['target']).sort_values(by=1.0,ascending=False)
Relevant_Exp_crosstab["Job Changers' Percentage (%)"]=round(Relevant_Exp_crosstab[1]/(Relevant_Exp_crosstab[0]+
                                                                                      Relevant_Exp_crosstab[1])*100,2)
Relevant_Exp_crosstab

In [ ]:
# Let's plot the result and visualize.

g=Relevant_Exp_crosstab.drop(["Job Changers' Percentage (%)"], axis=1).plot(kind='bar',stacked=False,figsize=(20,6))
for p in g.patches:
    g.annotate(format(p.get_height()), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center',
                   fontsize=14,
                   xytext = (0, 5), 
                   textcoords = 'offset points',
                   rotation='horizontal')
plt.xticks(rotation = 0,fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Revelent Experience',fontsize=16)
plt.ylabel('Target',fontsize=16)
plt.title('HR Analysis w.r.t Revelent Experience',fontsize=18)
plt.legend(['Not Looking for a Job','Looking for a Job'],title='Job Change Likelihood',title_fontsize=16,prop={"size":14})

* Most of the Job Changers are having relevent experience.

In [ ]:
#Enrolled University

df_train.enrolled_university.value_counts()

In [ ]:
# Dependancy of Enrolled University on Target Variable

University_crosstab=pd.crosstab(index=df_train['enrolled_university'],columns=df_train['target']).sort_values(by=1.0,ascending=False)
University_crosstab["Job Changers' Percentage (%)"]=round(University_crosstab[1]/(University_crosstab[0]+University_crosstab[1])*100,2)
University_crosstab

In [ ]:
# Let's plot the result and visualize.

g=University_crosstab.drop(["Job Changers' Percentage (%)"], axis=1).plot(kind='bar',stacked=False,figsize=(20,6))
for p in g.patches:
    g.annotate(format(p.get_height()), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center',
                   fontsize=14,
                   xytext = (0, 5), 
                   textcoords = 'offset points',
                   rotation='horizontal')
plt.xticks(rotation = 0,fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('University Enrollment',fontsize=16)
plt.ylabel('Target',fontsize=16)
plt.title('HR Analysis w.r.t University Enrollment',fontsize=18)
plt.legend(['Not Looking for a Job','Looking for a Job'],title='Job Change Likelihood',title_fontsize=16,prop={"size":14})

* Most of the Job Changers have not enrolled in a University!.

In [ ]:
# Education Level

df_train.education_level.value_counts()

In [ ]:
# Dependancy of Education on Target Variable

Education_crosstab=pd.crosstab(index=df_train['education_level'],columns=df_train['target']).sort_values(by=1.0,ascending=False)
Education_crosstab["Job Changers' Percentage (%)"]=round(Education_crosstab[1]/(Education_crosstab[0]+Education_crosstab[1])*100,2)
print(Education_crosstab)


# Let's plot the result and visualize.

g=Education_crosstab.drop(["Job Changers' Percentage (%)"], axis=1).plot(kind='bar',stacked=False,figsize=(20,6))
for p in g.patches:
    g.annotate(format(p.get_height()), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center',
                   fontsize=14,
                   xytext = (0, 5), 
                   textcoords = 'offset points',
                   rotation='horizontal')
plt.xticks(rotation = 0,fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Education Level',fontsize=16)
plt.ylabel('Target',fontsize=16)
plt.title('HR Analysis w.r.t Education Level',fontsize=18)
plt.legend(['Not Looking for a Job','Looking for a Job'],title='Job Change Likelihood',title_fontsize=16,prop={"size":14})

* Most of them trying to search for new Jobs are college graduates and the employees having a primary school level of education are the least Job changers.

In [ ]:
# Major Discipline

df_train.major_discipline.value_counts()

In [ ]:
# Dependancy of Major Discipline on Target Variable

Major_crosstab=pd.crosstab(index=df_train['major_discipline'],columns=df_train['target']).sort_values(by=1.0,ascending=False)
Major_crosstab["Job Changers' Percentage (%)"]=round(Major_crosstab[1]/(Major_crosstab[0]+Major_crosstab[1])*100,2)
Major_crosstab


# Let's plot the result and visualize.

g=Major_crosstab.drop(["Job Changers' Percentage (%)"], axis=1).plot(kind='bar',stacked=False,figsize=(20,6))
for p in g.patches:
    g.annotate(format(p.get_height()), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center',
                   fontsize=14,
                   xytext = (0, 6), 
                   textcoords = 'offset points',
                   rotation='horizontal')
plt.xticks(rotation = 0,fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Major Discipline',fontsize=16)
plt.ylabel('Target',fontsize=16)
plt.title('HR Analysis w.r.t Major Discipline',fontsize=18)
plt.legend(['Not Looking for a Job','Looking for a Job'],title='Job Change Likelihood',title_fontsize=16,prop={"size":14})

* Employees who had chosen STEM as their major descipline during college are the majority job changers.

In [ ]:
# Experience

df_train.experience.value_counts()

In [ ]:
# Dependancy of Experience on Target Variable

Experience_in_years_crosstab=pd.crosstab(index=df_train['experience'],columns=df_train['target']).sort_values(by=1.0,ascending=False)
Experience_in_years_crosstab["Job Changers' Percentage (%)"]=round(Experience_in_years_crosstab[1]/(Experience_in_years_crosstab[0]+
                                                                                                    Experience_in_years_crosstab[1])*100,2)
print(Experience_in_years_crosstab)


# Let's plot the result and visualize.

g=Experience_in_years_crosstab.drop(["Job Changers' Percentage (%)"], axis=1).plot(kind='bar',stacked=False,figsize=(19,6))
for p in g.patches:
    g.annotate(format(p.get_height()), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center',
                   fontsize=14,
                   xytext = (0, 20), 
                   textcoords = 'offset points',
                   rotation='vertical')
plt.xticks(rotation = 0,fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Experience',fontsize=16)
plt.ylabel('Target',fontsize=16)
plt.title('HR Analysis w.r.t Experience',fontsize=18)
plt.legend(['Not Looking for a Job','Looking for a Job'],title='Job Change Likelihood',title_fontsize=16,prop={"size":14})

* Employees having 20+ years of Experience are the most Job changers and Surprisingly,there are less Job changers with Experience between 10 to 20 years.

In [ ]:
#Company Size

df_train.company_size.value_counts()

In [ ]:
# Dependancy of Company Size on Target Variable

Company_Size_crosstab=pd.crosstab(index=df_train['company_size'],columns=df_train['target']).sort_values(by=1.0,ascending=False)
Company_Size_crosstab["Job Changers' Percentage (%)"]=round(Company_Size_crosstab[1]/(Company_Size_crosstab[0]+
                                                                                      Company_Size_crosstab[1])*100,2)
print(Company_Size_crosstab)


# Let's plot the result and visualize.

g=Company_Size_crosstab.drop(["Job Changers' Percentage (%)"], axis=1).plot(kind='bar',stacked=False,figsize=(19,6))
for p in g.patches:
    g.annotate(format(p.get_height()), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center',
                   fontsize=14,
                   xytext = (0, 6), 
                   textcoords = 'offset points',
                   rotation='horizontal')
plt.xticks(rotation = 0,fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Company Size',fontsize=16)
plt.ylabel('Target',fontsize=16)
plt.title('HR Analysis w.r.t Company Size',fontsize=18)
plt.legend(['Not Looking for a Job','Looking for a Job'],title='Job Change Likelihood',title_fontsize=16,prop={"size":14})

In [ ]:
#Last New Job

df_train.last_new_job.value_counts()

In [ ]:
# Dependancy of Last New Job on Target Variable

Last_New_Job_crosstab=pd.crosstab(index=df_train['last_new_job'],columns=df_train['target']).sort_values(by=1.0,ascending=False)
Last_New_Job_crosstab["Job Changers' Percentage (%)"]=round(Last_New_Job_crosstab[1]/(Last_New_Job_crosstab[0]+
                                                                                      Last_New_Job_crosstab[1])*100,2)
print(Last_New_Job_crosstab)


# Let's plot the result and visualize.

g=Last_New_Job_crosstab.drop(["Job Changers' Percentage (%)"], axis=1).plot(kind='bar',stacked=False,figsize=(20,6))
for p in g.patches:
    g.annotate(format(p.get_height()), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center',
                   fontsize=14,
                   xytext = (0, 6), 
                   textcoords = 'offset points',
                   rotation='horizontal')
plt.xticks(rotation = 0,fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Last new Job',fontsize=16)
plt.ylabel('Target',fontsize=16)
plt.title('HR Analysis w.r.t Last new Job',fontsize=18)
plt.legend(['Not Looking for a Job','Looking for a Job'],title='Job Change Likelihood',title_fontsize=16,prop={"size":14})

* Employees having 1 year Experience in the Current Company are the most Job Changers.

In [ ]:
# Let's analyze the No of training hrs of the Employees.

plt.figure(figsize=(18,5))
sns.histplot(df_train.training_hours,bins=350)
plt.xticks(rotation = 0,fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Training Hours',fontsize=16)
plt.ylabel('Distribution',fontsize=16)
plt.title('Histogram plot of Employee Training Hours',fontsize=18)

* Most Employees have undergone about 50 hrs of training and some of them have undergone Extensive training of more than 100 hrs.

In [ ]:
# Checking for outliers.

plt.figure(figsize=(12,5))
sns.boxplot(x=df_train.training_hours)
plt.xticks(rotation = 0,fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Training Hours',fontsize=16)
plt.ylabel('Distribution',fontsize=16)
plt.title('Box Plot of Employee Training Hours',fontsize=18)

In [ ]:
x_random = np.random.normal(0, 2, 10000)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(18,6))

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('No of Hours',fontsize=14)
plt.ylabel('Probability Distribution',fontsize=14)

ax1.hist(x_random, bins='auto')
ax1.set_title('probability density (Random Data)',fontsize=18)

ax2.hist(df_train.training_hours, bins='auto')
ax2.set_title('Training Hours',fontsize=18)

* **The Training_hours feature is negatively Skewed.**
* **But, after applying log & square root transformations, the data was almost positively skewed but, the Score of the feature went down.**
* **So, it's better to keep it in it's original form**

In [ ]:
# Let's partition the data into "Normal", "Extensive" & "Rigorous" Training

labels=['Normal Training','Extensive Training','Rigorous Training']

df_train['Training_length']=pd.cut(df_train.training_hours,[0,100,200,350],right=False,labels=labels)

In [ ]:
# Dependancy of Training Length on Target Variable

Training_length_crosstab=pd.crosstab(index=df_train['Training_length'],columns=df_train['target']).sort_values(by=1.0,ascending=False)
Training_length_crosstab["Job Changers' Percentage (%)"]=round(Training_length_crosstab[1]/(Training_length_crosstab[0]+
                                                                                      Training_length_crosstab[1])*100,2)
print(Training_length_crosstab)


# Let's plot the result and visualize.

g=Training_length_crosstab.drop(["Job Changers' Percentage (%)"], axis=1).plot(kind='bar',stacked=False,figsize=(20,6))
for p in g.patches:
    g.annotate(format(p.get_height()), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center',
                   fontsize=14,
                   xytext = (0, 6), 
                   textcoords = 'offset points',
                   rotation='horizontal')
plt.xticks(rotation = 0,fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel('Training Length',fontsize=16)
plt.ylabel('Target',fontsize=16)
plt.title('HR Analysis w.r.t Training Length',fontsize=18)
plt.legend(['Not Looking for a Job','Looking for a Job'],title='Job Change Likelihood',title_fontsize=16,prop={"size":14})

* Most of the Job Changing Employees have undergone training for less than 100 hrs.

* **Low City development for CDI scores between 0.0 and 0.5.**
* **Medium City development for CDI scores between 0.5 and 0.8.**
* **High City development for CDI scores between 0.8 and 1.0.**

In [ ]:
# Let's analyze the Probability Distribution of City development index through a histogram.

plt.figure(figsize=(18,6))
sns.histplot(x=df_train.city_development_index,kde=True)
plt.xticks(rotation = 0,fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('City Development Index',fontsize=16)
plt.ylabel('Distribution',fontsize=16)
plt.title('Histogram plot of City Development Index',fontsize=18)

* **Most of the Employees live in cities with CDI greater than 0.9 which indicates High development.**

In [ ]:
# Let's round off the values of CDI to 3 decimals.

df_train.city_development_index=df_train.city_development_index.apply(lambda x: round(x,3))

In [ ]:
plt.figure(figsize=(20,6))
sns.barplot(x='city',y='city_development_index',data=df_train.sort_values(by='city_development_index',ascending=False))
plt.xticks(rotation = 90,fontsize=10)
plt.yticks(fontsize=16)
plt.xlabel('Cities',fontsize=16)
plt.ylabel('City Development Index',fontsize=16)
plt.title('Cities w.r.t City Development Index Level',fontsize=18)

* **Almost All cities have a CDI above 0.4.**
* **Let's Categorize the CDI to "Low Development", "Medium Development" & "High Development" for clear understanding.**

In [ ]:
def update_CDI(index):
    if(index >= 0 and index <= 0.5):
        return "Less Development"
    elif(index > 0.5 and index <= 0.7):
        return "Moderate Development"
    elif(index > 0.7):
        return "High Developement"

In [ ]:
# Let's categorize the City development level into Less, Moderate & High Development.

df_train["City_development_level"]=df_train.city_development_index.apply(update_CDI)
df_train.City_development_level.value_counts()

In [ ]:
# Dependancy of City Development Level on Target Variable

CDI_crosstab=pd.crosstab(index=df_train['City_development_level'],columns=df_train['target']).sort_values(by=1.0,ascending=False)
CDI_crosstab["Job Changers' Percentage (%)"]=round(CDI_crosstab[1]/(CDI_crosstab[0]+CDI_crosstab[1])*100,2)
print(CDI_crosstab)


# Let's plot the result and visualize.

g=CDI_crosstab.drop(["Job Changers' Percentage (%)"], axis=1).plot(kind='bar',stacked=False,figsize=(20,6))
for p in g.patches:
    g.annotate(format(p.get_height()), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center',
                   fontsize=14,
                   xytext = (0, 6), 
                   textcoords = 'offset points',
                   rotation='horizontal')
plt.xticks(rotation = 0,fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel('City Development Index Level',fontsize=16)
plt.ylabel('Target',fontsize=16)
plt.title('HR Analysis w.r.t City Development Index Level',fontsize=18)
plt.legend(['Not Looking for a Job','Looking for a Job'],title='Job Change Likelihood',title_fontsize=16,prop={"size":14})

* **Most of the Job changers live in Moderately developed cities.**

# Data Pre-processing

In [ ]:
# Label Encoding

df_train.education_level=df_train.education_level.map({"Primary School":0,"High School":1,"Graduate":2,"Masters":3,"Phd":4})

df_train.experience=df_train.experience.map({"<1":0,"1":1,"2":2,"3":3,"4":4,"5":5,"6":6,"7":7,"8":8,"9":9,"10":10,"11":11,"12":12,"13":13,
                                             '14':14,"15":15,"16":16,"17":17,"18":18,"19":19,"20":20,">20":21})

df_train.City_development_level=df_train.City_development_level.map({"Less Development":0,"Moderate Development":1,"High Developement":2})

df_train.gender=df_train.gender.map({'Male':0,'Female':1,'Other':2})

df_train.enrolled_university=df_train.enrolled_university.map({'Full time course':0,'Part time course':1,'no_enrollment':2})

df_train.major_discipline=df_train.major_discipline.map({'STEM':0,'Humanities':1,'Business Degree':2,'Arts':3,'Other':4,'No Major':5})

df_train.company_size=df_train.company_size.map({'10000+':0,'5000-9999':1,'1000-4999':2,'500-999':3,'100-500':4,'50-99':5,'10/49':6,
                                                '<10':7})

df_train.company_type=df_train.company_type.map({'Pvt Ltd':0,'Public Sector':1,'Funded Startup':2,'Early Stage Startup':3,'NGO':4,
                                                'Other':5})

df_train.last_new_job=df_train.last_new_job.map({'>4':0,'4':1,'3':2,'2':3,'1':4,'never':5})

df_train.Training_length=df_train.Training_length.map({'Rigorous Training':0,'Extensive Training':1,'Normal Training':2})



# Let's apply one-hot Encoding to relevent_experience & city Features.

df_train=pd.get_dummies(data=df_train,columns=['relevent_experience','city'],drop_first=True)


# let's drop the Employee ID field

df_train.drop(['enrollee_id'],axis=1,inplace=True)

In [ ]:
X_temp=df_train.drop('target',axis=1)
y_temp=df_train.target
print(X_temp.columns)

In [ ]:
X_temp.rename(columns={'company_size_<10':'company_size_lessthan10','last_new_job_>4':'last_new_job_greaterthan4'},inplace=True)

# Feature Selection

* Let's use SelectKBest function to obtain scores of all the Dataset features.

In [ ]:
Best_Params=SelectKBest(score_func=chi2, k='all')
Best_Params.fit(X_temp,y_temp)

In [ ]:
df_scores=pd.DataFrame(Best_Params.scores_)
df_columns=pd.DataFrame(X_temp.columns)
df_scores_evaluation=pd.concat([df_scores,df_columns],axis=1)
df_scores_evaluation.columns=['Scores','Features']
df_scores_evaluation=df_scores_evaluation.nlargest(131,'Scores')
print(df_scores_evaluation)

In [ ]:
X_new=copy.deepcopy(X_temp)
print(len(X_temp.columns))
print(len(X_new.columns))
print(X_new.columns)

y_new=copy.deepcopy(y_temp)

In [ ]:
# Let's split the data into Training & Test sets.

X_train,X_test,y_train,y_test=train_test_split(X_new,y_new,test_size=0.20,random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# Checking Target Class Imbalance

plt.figure(figsize=(8,5))
g=sns.countplot(x="target",data=df_train)
for p in g.patches:
    g.annotate(format(p.get_height()), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center',
                   fontsize=12,
                   xytext = (0, 6), 
                   textcoords = 'offset points',
                   rotation='horizontal')
plt.xticks(rotation = 0,fontsize=16)
plt.yticks(fontsize=18)
plt.xlabel('Target Classes',fontsize=16)
plt.ylabel('Count',fontsize=16)
plt.title('Target Class Imbalance',fontsize=18)
plt.legend(['Not Looking for a Job','Looking for a Job'],title='Target Class',title_fontsize=15,prop={"size":14})

* **As the Target label is highly Imbalanced, we need to upsample the lower class (which is 1.0 in this case).**
* **If we downsample the Majority class, we might lose significant data.**
* **We can also use SMOTE (Synthetic Minority Oversampling Technique) which creates synthetic (duplicate) data points taken from the Majority class and can be very effective.**
* **We can apply both resampling methods and compare the results.**

In [ ]:
# SMOTE

print("Counts of Label 0 before Oversampling: {}".format(sum(y_train == 0.0)))

print("Counts of Label 1 before Oversampling: {}".format(sum(y_train == 1.0)))

smote=SMOTE(random_state=0)

X_train_smote, y_train_smote=smote.fit_resample(X_train,y_train)

print("Shape of X_train after Oversampling: {}".format(X_train_smote.shape))

print("Shape of y_train after Oversampling: {}".format(y_train_smote.shape))

print("Counts of Label 0 after Oversampling: {}".format(sum(y_train_smote==0.0)))

print("Counts of Label 1 after Oversampling: {}".format(sum(y_train_smote==1.0)))

In [ ]:
# Let's use resample method from sklearn library to Oversample the minority class.

X=pd.concat([X_train,y_train],axis=1)

Emp_NotLeaving=X[X.target == 0]
Emp_Leaving=X[X.target == 1]

upsampled_Emp_Leaving=resample(Emp_Leaving,
                       replace=True,
                       n_samples=len(Emp_NotLeaving),
                       random_state=42)

upsampled_Data=pd.concat([Emp_NotLeaving,upsampled_Emp_Leaving])


y_train_upsampled=upsampled_Data.target
X_train_upsampled=upsampled_Data.drop('target',axis=1)


print(len(y_train_upsampled == 1))
print(len(y_train_upsampled == 0))

In [ ]:
# Let's convert all the Dataset feature names into list so that we can pass the desired set of features to the models separately for 
#training. 

df_scores_evaluation=df_scores_evaluation.Features.to_list()

In [ ]:
# From the SelectKBest method, I have obtained the scores for all features & from trial & error technique, I have determined the 
# Number of features to pass to each model for obtaining the best results.
# So, I'll be dropping the respective number of features below.

lr_ridge_features=df_scores_evaluation[55:]    # dropping all features except the top 55.
svm_features=df_scores_evaluation[90:]
dt_features=df_scores_evaluation[100:]
bc_lgbm_features=df_scores_evaluation[30:]
rf_features=df_scores_evaluation[120:]

print(len(lr_ridge_features))
print(len(svm_features))
print(len(dt_features))
print(len(bc_lgbm_features))
print(len(rf_features))

In [ ]:
# I'm dropping the above mentioned features explicitely from the dataset & passing the best set of features to each model.

lr_ridge_X_train=copy.deepcopy(X_train_upsampled)
lr_ridge_X_test=copy.deepcopy(X_test)
lr_ridge_X_train.drop(lr_ridge_features,axis=1,inplace=True)
lr_ridge_X_test.drop(lr_ridge_features,axis=1,inplace=True)

svm_X_train=copy.deepcopy(X_train_smote)
svm_X_test=copy.deepcopy(X_test)
svm_X_train.drop(svm_features,axis=1,inplace=True)
svm_X_test.drop(svm_features,axis=1,inplace=True)

dt_X_train=copy.deepcopy(X_train_upsampled)
dt_X_test=copy.deepcopy(X_test)
dt_X_train.drop(dt_features,axis=1,inplace=True)
dt_X_test.drop(dt_features,axis=1,inplace=True)

bc_lgbm_X_train=copy.deepcopy(X_train_smote)
bc_lgbm_X_test=copy.deepcopy(X_test)
bc_lgbm_X_train.drop(bc_lgbm_features,axis=1,inplace=True)
bc_lgbm_X_test.drop(bc_lgbm_features,axis=1,inplace=True)

rf_X_train=copy.deepcopy(X_train_upsampled)
rf_X_test=copy.deepcopy(X_test)
rf_X_train.drop(rf_features,axis=1,inplace=True)
rf_X_test.drop(rf_features,axis=1,inplace=True)

In [ ]:
# LOGISTIC REGRESSION MODEL

lr_model=LogisticRegression()

t0=time.time()

# Hyper-parameter tuning
#solvers = ['newton-cg', 'lbfgs', 'liblinear', 'saga']
#penalty = ['l2']
#lr_c = [100, 10, 1.0, 0.1, 0.01]

solvers = ['newton-cg']
penalty=['l2']
lr_c = [10]

lr_grid = dict(solver=solvers,penalty=penalty,C=lr_c)

# cross-validation using Repeated Stratified K-fold method.
lr_cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# Grid Search CV method loops through the different hyper parameters determining the optimal values.
lr_grid_search = GridSearchCV(estimator=lr_model, param_grid=lr_grid,verbose=1, cv=lr_cv, n_jobs=-1, scoring='accuracy',error_score=0)

# Fitting the Model to the Dataset.
lr_grid_result=lr_grid_search.fit(lr_ridge_X_train,y_train_upsampled)

print("training time:", round(time.time()-t0, 3), "s")

# returns the best hyper parameters.
#r_grid_result.best_params_

In [ ]:
# Making predictions using our model.
lr_grid_predictions=lr_grid_result.predict(lr_ridge_X_test)

In [ ]:
print("LOGISTIC REGRESSION Model Performance Metrics:")
print(classification_report(y_test,lr_grid_predictions))

#print(classification_report(Test_Label,lr_grid_result_test_data_predictions))

print("CONFUSION MATRIX :")
print(confusion_matrix(y_test,lr_grid_predictions))
#print(confusion_matrix(Test_Label,lr_grid_result_test_data_predictions))
print("\n")

plt.figure(figsize=(12,6))
plt.plot([0, 1], [0, 1], linestyle='--', label='No Skill Classifier') # Let's define a no skill (Dummy) Classifier for reference.

fpr, tpr, _ = roc_curve(y_test,lr_grid_predictions)             # passing the target labels & Model Predictions to the roc_curve method. 

#fpr, tpr, _ = roc_curve(Test_Label,lr_grid_result_test_data_predictions)

plt.plot(fpr, tpr, marker='.', label='Logistic Regression')  # Plotting the obtained results.

ns_probs = [0 for _ in range(len(y_test))]       # generating predictions from the no skill (Dummy) classifier for reference.

#ns_probs = [0 for _ in range(len(Test_Label))]

# Calculating Area under Curve for the No Skill & the trained Model.
ns_auc = roc_auc_score(y_test, ns_probs)

#ns_auc = roc_auc_score(Test_Label, ns_probs)

lr_auc = roc_auc_score(y_test, lr_grid_predictions)

#lr_auc = roc_auc_score(Test_Label, lr_grid_result_test_data_predictions)

print('NO SKILL CLASSIFIER: ROC AUC=%.3f' % (ns_auc))
print('LOGISTIC REGRESSION: ROC AUC=%.3f' % (lr_auc))

plt.xlabel('False Positive Rate',fontsize=16)
plt.ylabel('True Positive Rate',fontsize=16)
plt.title("ROC Curve",fontsize=18)
plt.legend(prop={'size':12})
plt.show()


# Let's plot the Precision-Recall Curve

precision, recall, thresholds = precision_recall_curve(y_test, lr_grid_predictions)
lr_auc_score = auc(recall, precision)

no_skill = len(y_test[y_test==1]) / len(y_test)
plt.title('Precision-Recall Curve',fontsize=18)
plt.plot([0, 1], [0.5, 0.5], linestyle='--')
plt.plot(recall, precision, marker='.')
print('Precision-Recall AUC Score: %.3f' % lr_auc_score)
plt.show()

In [ ]:
rc_model=RidgeClassifier()

t0=time.time()

#alpha = [0.2, 0.4, 0.6, 0.8, 1.0]

alpha=[0.8]

rc_grid = dict(alpha=alpha)

rc_cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

rc_grid_search = GridSearchCV(estimator=rc_model, param_grid=rc_grid,verbose=1, cv=rc_cv, scoring='accuracy',error_score=0)

rc_grid_result=rc_grid_search.fit(lr_ridge_X_train,y_train_upsampled)

print("training time:", round(time.time()-t0, 3), "s")

#rc_grid_result.best_params_

In [ ]:
rc_grid_predictions=rc_grid_result.predict(lr_ridge_X_test)

In [ ]:
print("RIDGE CLASSIFIER Model Performance Metrics:")
print(classification_report(y_test,rc_grid_predictions))

print("CONFUSION MATRIX :")
print(confusion_matrix(y_test,rc_grid_predictions))
print("\n")

plt.figure(figsize=(12,6))
plt.plot([0, 1], [0, 1], linestyle='--', label='No Skill Classifier') # Let's define a no skill (Dummy) Classifier for reference.

fpr, tpr, _ = roc_curve(y_test,rc_grid_predictions)             # passing the target labels & Model Predictions to the roc_curve method. 

plt.plot(fpr, tpr, marker='.', label='RIDGE CLASSIFIER')  # Plotting the obtained results.

ns_probs = [0 for _ in range(len(y_test))]       # generating predictions from the no skill (Dummy) classifier for reference.

# Calculating Area under Curve for the No Skill & the trained Model.
ns_auc = roc_auc_score(y_test, ns_probs)             
rc_auc = roc_auc_score(y_test, rc_grid_predictions)

print('NO SKILL CLASSIFIER: ROC AUC=%.3f' % (ns_auc))
print('RIDGE CLASSIFIER: ROC AUC=%.3f' % (rc_auc))

plt.xlabel('False Positive Rate',fontsize=16)
plt.ylabel('True Positive Rate',fontsize=16)
plt.title("ROC Curve",fontsize=18)
plt.legend(prop={'size':12})
plt.show()

# Let's plot the Precision-Recall Curve

precision, recall, thresholds = precision_recall_curve(y_test, rc_grid_predictions)
rc_auc_score = auc(recall, precision)

no_skill = len(y_test[y_test==1]) / len(y_test)
plt.title('Precision-Recall Curve',fontsize=18)
plt.plot([0, 1], [0.5, 0.5], linestyle='--')
plt.plot(recall, precision, marker='.')
print('Precision-Recall AUC Score: %.3f' % rc_auc_score)
plt.show()

In [ ]:
svm_model=LinearSVC()

t0=time.time()

#svm_C=[0.8,1,2,4]

svm_C=[2]
svc_class_weight=['balanced']
dual=[False]

svm_grid=dict(C=svm_C,class_weight=svc_class_weight,dual=dual)

svm_cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

svm_grid_search = GridSearchCV(estimator=svm_model, param_grid=svm_grid, cv=svm_cv, scoring='accuracy',error_score=0)

svm_grid_result=svm_grid_search.fit(svm_X_train,y_train_smote) 

print("training time:", round(time.time()-t0, 3), "s")

#svm_grid_result.best_params_

In [ ]:
svm_grid_predictions=svm_grid_result.predict(svm_X_test)

In [ ]:
print("SVM CLASSIFIER Model Performance Metrics:")
print(classification_report(y_test,svm_grid_predictions))

print("CONFUSION MATRIX :")
print(confusion_matrix(y_test,svm_grid_predictions))
print("\n")

plt.figure(figsize=(12,6))
plt.plot([0, 1], [0, 1], linestyle='--', label='No Skill Classifier') # Let's define a no skill (Dummy) Classifier for reference.

fpr, tpr, _ = roc_curve(y_test,svm_grid_predictions)             # passing the target labels & Model Predictions to the roc_curve method. 

plt.plot(fpr, tpr, marker='.', label='SVM CLASSIFIER')  # Plotting the obtained results.

ns_probs = [0 for _ in range(len(y_test))]       # generating predictions from the no skill (Dummy) classifier for reference.

# Calculating Area under Curve for the No Skill & the trained Model.
ns_auc = roc_auc_score(y_test, ns_probs)             
svm_auc = roc_auc_score(y_test, svm_grid_predictions)

print('NO SKILL CLASSIFIER: ROC AUC=%.3f' % (ns_auc))
print('SVM CLASSIFIER: ROC AUC=%.3f' % (svm_auc))

plt.xlabel('False Positive Rate',fontsize=16)
plt.ylabel('True Positive Rate',fontsize=16)
plt.title("ROC Curve",fontsize=18)
plt.legend(prop={'size':12})
plt.show()


# Let's plot the Precision-Recall Curve

precision, recall, thresholds = precision_recall_curve(y_test, svm_grid_predictions)
svm_auc_score = auc(recall, precision)

no_skill = len(y_test[y_test==1]) / len(y_test)
plt.title('Precision-Recall Curve',fontsize=18)
plt.plot([0, 1], [0.5, 0.5], linestyle='--')
plt.plot(recall, precision, marker='.')
print('Precision-Recall AUC Score: %.3f' % svm_auc_score)
plt.show()

In [ ]:
dtc_model=DecisionTreeClassifier()

t0=time.time()

#dtc_criterion=['gini','entropy']
#dtc_max_depth=[14,16,18,20,24,28]
#dtc_max_features = ['sqrt', 'log2']
#dtc_min_samples_split = [6,8,10,12]
#dtc_min_samples_leaf = [1, 2, 5, 10]

dtc_criterion=['gini']
dtc_max_depth=[12]
dtc_max_features = ['sqrt']
dtc_min_samples_split = [8]
dtc_min_samples_leaf = [1]
dtc_class_weight=['balanced']


dtc_grid=dict(criterion=dtc_criterion,max_depth=dtc_max_depth,max_features=dtc_max_features,min_samples_split=dtc_min_samples_split,
             min_samples_leaf=dtc_min_samples_leaf,class_weight=dtc_class_weight)

dtc_cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

dtc_grid_search = GridSearchCV(estimator=dtc_model, param_grid=dtc_grid, cv=dtc_cv,n_jobs=-1,verbose=1, scoring='accuracy',error_score=0)

dtc_grid_result=dtc_grid_search.fit(dt_X_train,y_train_upsampled) 

print("training time:", round(time.time()-t0, 3), "s")

#dtc_grid_result.best_params_

In [ ]:
dtc_grid_predictions=dtc_grid_result.predict(dt_X_test)

In [ ]:
print("DECISION TREE CLASSIFIER Model Performance Metrics:")
print(classification_report(y_test,dtc_grid_predictions))

print("CONFUSION MATRIX :")
print(confusion_matrix(y_test,dtc_grid_predictions))
print("\n")

plt.figure(figsize=(12,6))
plt.plot([0, 1], [0, 1], linestyle='--', label='No Skill Classifier') # Let's define a no skill (Dummy) Classifier for reference.

fpr, tpr, _ = roc_curve(y_test,dtc_grid_predictions)             # passing the target labels & Model Predictions to the roc_curve method. 

plt.plot(fpr, tpr, marker='.', label='DECISION TREE CLASSIFIER')  # Plotting the obtained results.

ns_probs = [0 for _ in range(len(y_test))]       # generating predictions from the no skill (Dummy) classifier for reference.

# Calculating Area under Curve for the No Skill & the trained Model.
ns_auc = roc_auc_score(y_test, ns_probs)             
dtc_auc = roc_auc_score(y_test, dtc_grid_predictions)

print('NO SKILL CLASSIFIER: ROC AUC=%.3f' % (ns_auc))
print('DECISION TREE CLASSIFIER: ROC AUC=%.3f' % (dtc_auc))

plt.xlabel('False Positive Rate',fontsize=16)
plt.ylabel('True Positive Rate',fontsize=16)
plt.title("ROC Curve",fontsize=18)
plt.legend(prop={'size':12})
plt.show()


# Let's plot the Precision-Recall Curve

precision, recall, thresholds = precision_recall_curve(y_test, dtc_grid_predictions)
dtc_auc_score = auc(recall, precision)

no_skill = len(y_test[y_test==1]) / len(y_test)
plt.title('Precision-Recall Curve',fontsize=18)
plt.plot([0, 1], [0.5, 0.5], linestyle='--')
plt.plot(recall, precision, marker='.')
print('AUC Score: %.3f' % dtc_auc_score)
plt.show()

In [ ]:
#bc_model=BaggingClassifier()

bc_model=BalancedBaggingClassifier()

#print(bc_model.get_params().keys())

#bc_max_samples=[0.8,1.0,1.2,1.4]
#bc_max_features=[12,14,16,18]
#bc_base_estimator=KNeighborsClassifier()
#bc_bootstrap=[True,False]
#bc_bootstrap_features=[True,False]

t0=time.time()

bc_n_estimators = [1000]
bc_max_samples=[1.0]
bc_max_features=[12]
bc_bootstrap=[False]
bc_bootstrap_features=[False]

bc_grid = dict(n_estimators=bc_n_estimators,max_samples=bc_max_samples,max_features=bc_max_features,bootstrap=bc_bootstrap,
              bootstrap_features=bc_bootstrap_features)

bc_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)

bc_grid_search = GridSearchCV(estimator=bc_model, param_grid=bc_grid, n_jobs=-1, verbose=1, cv=bc_cv, scoring='accuracy',error_score=0)

bc_grid_result=bc_grid_search.fit(bc_lgbm_X_train,y_train_smote)

print("training time:", round(time.time()-t0, 3), "s")

bc_grid_result.best_params_

In [ ]:
bc_grid_predictions=bc_grid_result.predict(bc_lgbm_X_test)

In [ ]:
print("BAGGING CLASSIFIER Model Performance Metrics:")
print(classification_report(y_test,bc_grid_predictions))

print("CONFUSION MATRIX :")
print(confusion_matrix(y_test,bc_grid_predictions))
print("\n")

plt.figure(figsize=(12,6))
plt.plot([0, 1], [0, 1], linestyle='--', label='No Skill Classifier') # Let's define a no skill (Dummy) Classifier for reference.

fpr, tpr, _ = roc_curve(y_test,bc_grid_predictions)             # passing the target labels & Model Predictions to the roc_curve method. 

plt.plot(fpr, tpr, marker='.', label='BAGGING CLASSIFIER')  # Plotting the obtained results.

ns_probs = [0 for _ in range(len(y_test))]       # generating predictions from the no skill (Dummy) classifier for reference.

# Calculating Area under Curve for the No Skill & the trained Model.
ns_auc = roc_auc_score(y_test, ns_probs)             
bc_auc = roc_auc_score(y_test, bc_grid_predictions)

print('NO SKILL CLASSIFIER: ROC AUC=%.3f' % (ns_auc))
print('BAGGING CLASSIFIER: ROC AUC=%.3f' % (bc_auc))

plt.xlabel('False Positive Rate',fontsize=16)
plt.ylabel('True Positive Rate',fontsize=16)
plt.title("ROC Curve",fontsize=18)
plt.legend(prop={'size':12})
plt.show()


# Let's plot the Precision-Recall Curve

precision, recall, thresholds = precision_recall_curve(y_test, bc_grid_predictions)
bc_auc_score = auc(recall, precision)

no_skill = len(y_test[y_test==1]) / len(y_test)
plt.title('Precision-Recall Curve',fontsize=18)
plt.plot([0, 1], [0.5, 0.5], linestyle='--')
plt.plot(recall, precision, marker='.')
print('AUC Score: %.3f' % bc_auc_score)
plt.show()

In [ ]:
rfc_model=RandomForestClassifier()

t0=time.time()


#rfc_max_features = ['sqrt', 'log2']
#rfc_min_samples_split = [1,2,4,6,8]
#rfc_min_samples_leaf = [1, 2, 5, 10]
#rfc_max_depth = [10,12,14,16]


rfc_n_estimators = [1000]
rfc_max_features = ['sqrt']
rfc_max_depth = [12]
rfc_min_samples_split = [2]
rfc_min_samples_leaf = [1]
rfc_class_weight=['balanced']


#rfc_grid = dict(n_estimators=rfc_n_estimators,max_features=rfc_max_features,class_weight=rfc_class_weight,max_depth=rfc_max_depth,
#               min_samples_split=rfc_min_samples_split,min_samples_leaf=rfc_min_samples_leaf)

rfc_grid = dict(n_estimators=rfc_n_estimators,max_features=rfc_max_features,class_weight=rfc_class_weight,max_depth=rfc_max_depth,
               min_samples_split=rfc_min_samples_split,min_samples_leaf=rfc_min_samples_leaf)

rfc_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)

rfc_grid_search = GridSearchCV(estimator=rfc_model, param_grid=rfc_grid, n_jobs=-1, verbose=3, cv=rfc_cv, scoring='accuracy',error_score=0)

rfc_grid_result=rfc_grid_search.fit(rf_X_train,y_train_upsampled)

print("training time:", round(time.time()-t0, 3)/60)

#rfc_grid_result.best_params_

In [ ]:
rfc_grid_predictions=rfc_grid_result.predict(rf_X_test)

In [ ]:
print("RANDOM FOREST CLASSIFIER Model Performance Metrics:")
print(classification_report(y_test,rfc_grid_predictions))

#print(classification_report(Test_Label,rfc_grid_Test_Data_predictions))

print("CONFUSION MATRIX :")
print(confusion_matrix(y_test,rfc_grid_predictions))

#print(confusion_matrix(Test_Label,rfc_grid_Test_Data_predictions))

print("\n")

plt.figure(figsize=(12,6))
plt.plot([0, 1], [0, 1], linestyle='--', label='No Skill Classifier') # Let's define a no skill (Dummy) Classifier for reference.

fpr, tpr, _ = roc_curve(y_test,rfc_grid_predictions)             # passing the target labels & Model Predictions to the roc_curve method. 

#fpr, tpr, _ = roc_curve(Test_Label,rfc_grid_Test_Data_predictions)    

plt.plot(fpr, tpr, marker='.', label='RANDOM FOREST CLASSIFIER')  # Plotting the obtained results.

ns_probs = [0 for _ in range(len(y_test))]       # generating predictions from the no skill (Dummy) classifier for reference.

#ns_probs = [0 for _ in range(len(Test_Label))]

# Calculating Area under Curve for the No Skill & the trained Model.
ns_auc = roc_auc_score(y_test, ns_probs)

#ns_auc = roc_auc_score(Test_Label, ns_probs)

rfc_auc = roc_auc_score(y_test, rfc_grid_predictions)

#rfc_auc = roc_auc_score(Test_Label, rfc_grid_Test_Data_predictions)

print('NO SKILL CLASSIFIER: ROC AUC=%.3f' % (ns_auc))
print('RANDOM FOREST CLASSIFIER: ROC AUC=%.3f' % (rfc_auc))

plt.xlabel('False Positive Rate',fontsize=16)
plt.ylabel('True Positive Rate',fontsize=16)
plt.title("ROC Curve",fontsize=18)
plt.legend(prop={'size':12})
plt.show()


# Let's plot the Precision-Recall Curve

precision, recall, thresholds = precision_recall_curve(y_test, rfc_grid_predictions)
rfc_auc_score = auc(recall, precision)

no_skill = len(y_test[y_test==1]) / len(y_test)
plt.title('Precision-Recall Curve',fontsize=18)
plt.plot([0, 1], [0.5, 0.5], linestyle='--')
plt.plot(recall, precision, marker='.')
print('AUC Score: %.3f' % rfc_auc_score)
plt.show()

In [ ]:
lgbc_model=LGBMClassifier(is_balance=False)

t0=time.time()

#lgbc_n_estimators=[600,700,800,900]
#lgbc_max_depth=[10,12,14,16]
#lgbc_num_leaves=[8,10,12,14]
#lgbc_learning_rate=[0.0001, 0.001, 0.01, 0.1, 1.0]
#lgbc_boosting_type=['gbdt', 'dart', 'goss']
#min_child_samples=[100,200,300,400,500]
#min_child_weight=[1e-2, 1e-1, 1, 1e1, 1e2]

lgbc_n_estimators=[800]
lgbc_max_depth=[14]
lgbc_num_leaves=[14]
lgbc_learning_rate=[0.1]
lgbc_boosting_type=['gbdt']
objective=['binary']
min_child_samples=[100]
min_child_weight=[10]


lgbc_grid=dict(n_estimators=lgbc_n_estimators,max_depth=lgbc_max_depth,num_leaves=lgbc_num_leaves,learning_rate=lgbc_learning_rate,
              boosting_type=lgbc_boosting_type,objective=objective,min_child_samples=min_child_samples,min_child_weight=min_child_weight)

lgbc_cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)

lgbc_grid_search = GridSearchCV(estimator=lgbc_model, param_grid=lgbc_grid, n_jobs=-1,verbose=3, cv=lgbc_cv, scoring='accuracy',error_score=0)

lgbc_grid_result=lgbc_grid_search.fit(bc_lgbm_X_train,y_train_smote)

print("training time:", round(time.time()-t0, 3)/60)

#lgbc_grid_result.best_params_

In [ ]:
lgbc_grid_predictions=lgbc_grid_result.predict(bc_lgbm_X_test)

In [ ]:
print("GRADIENT BOOSTING Model Performance Metrics:")
print(classification_report(y_test,lgbc_grid_predictions))

print("CONFUSION MATRIX :")
print(confusion_matrix(y_test,lgbc_grid_predictions))
print("\n")

plt.figure(figsize=(12,6))
plt.plot([0, 1], [0, 1], linestyle='--', label='No Skill Classifier') # Let's define a no skill (Dummy) Classifier for reference.

fpr, tpr, _ = roc_curve(y_test,lgbc_grid_predictions)             # passing the target labels & Model Predictions to the roc_curve method. 

plt.plot(fpr, tpr, marker='.', label='EXTREME GRADIENT BOOSTING MODEL')  # Plotting the obtained results.

ns_probs = [0 for _ in range(len(y_test))]       # generating predictions from the no skill (Dummy) classifier for reference.

# Calculating Area under Curve for the No Skill & the trained Model.
ns_auc = roc_auc_score(y_test, ns_probs)
          
lgbc_auc = roc_auc_score(y_test, lgbc_grid_predictions)

print('NO SKILL CLASSIFIER: ROC AUC=%.3f' % (ns_auc))
print('GRADIENT BOOSTING MODEL: ROC AUC=%.3f' % (lgbc_auc))

plt.xlabel('False Positive Rate',fontsize=16)
plt.ylabel('True Positive Rate',fontsize=16)
plt.title("ROC Curve",fontsize=18)
plt.legend(prop={'size':12})
plt.show()


# Let's plot the Precision-Recall Curve

precision, recall, thresholds = precision_recall_curve(y_test, lgbc_grid_predictions)
lgbc_auc_score = auc(recall, precision)

no_skill = len(y_test[y_test==1]) / len(y_test)
plt.title('Precision-Recall Curve',fontsize=18)
plt.plot([0, 1], [0.5, 0.5], linestyle='--')
plt.plot(recall, precision, marker='.')
print('AUC Score: %.3f' % lgbc_auc_score)
plt.show()

In [ ]:
# Model Names
Model_Names=['LOGISTIC REGRESSION', 'RIDGE CLASSIFIER', 'LINEAR SVM CLASSIFIER', 'DECISION TREE CLASSIFIER', 'BAGGING CLASSIFIER',
            'RANDOM FOREST CLASSIFIER', 'LIGHT GBM']

# DataFrame Index values
Index=['Accuracy (%)','Precision (%)','Recall (%)','F1 Score (%)', 'ROC AUC Score', 'P-R AUC Score']

# Model Prediction values
Model_Predictions=[lr_grid_predictions,rc_grid_predictions,svm_grid_predictions,dtc_grid_predictions,bc_grid_predictions,
                   rfc_grid_predictions,lgbc_grid_predictions]

ROC_scores=[lr_auc,rc_auc,svm_auc,dtc_auc,bc_auc,rfc_auc,lgbc_auc]

PR_scores=[lr_auc_score,rc_auc_score,svm_auc_score,dtc_auc_score,bc_auc_score,rfc_auc_score,lgbc_auc_score]

# Model Metrics methods
model_metrics=[accuracy_score,precision_score,recall_score,f1_score]

# DataFrame Initialisation
Model_Metrics_Comparison=pd.DataFrame(columns=Model_Names,index=Index)

# Let's fill the dataframe with the model metrics values of all the trained models above.
for index,metric in zip(range(0,4),model_metrics):
    for model_name,model_prediction in zip(Model_Names,Model_Predictions):
        Model_Metrics_Comparison[model_name].values[index]=metric(y_test,model_prediction)*100
        
for model_name, rfc_score,pr_score in zip(Model_Names,ROC_scores,PR_scores):
        Model_Metrics_Comparison[model_name].values[4]=rfc_score
        Model_Metrics_Comparison[model_name].values[5]=pr_score

# Metric values in Percentage (%).
Model_Metrics_Comparison

* Best model from the above in terms of accuracy is random Forest followed by bagging & Ridge Classifier.

* Based on Precision & recall (which I tried so much to improve but couldn't :( So someone please help), Random forest is some what   better than the rest followed by Ridge & Bagging Classifier.

* Based on the ROC AUC Scores, again Random forest, Ridge or Logistic Regression could be considered.

* But when dealing with an Imbalanced Dataset such as this, we should actually consider the precision-recall scores which accurately         incicate False +ve & False -ve values predicted by the models. 

* Based on the Precision-Recall AUC Scores, Random forest is the best model followed by Ridgr & bagging classifier.

**If you like my Kernel, Please Upvote. Please feel free to provide suggestions in the comments which helps me to improve myself. Thank you :)**